In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplhep as hep
hep.style.use('CMS')

from scipy.optimize import minimize
from pathlib import Path
from natsort import natsorted
from cycler import cycler
color_cycle =  ['#3f90da','#ffa90e','#bd1f01','#94a4a2','#832db6','#a96b59','#e76300','#b9ac70','#717581','#92dadd']

In [ ]:
files = natsorted(Path('./Run_PTNH1_230_OFF20_RFSel0_longrun_1').glob('*csv'))

In [ ]:
df = pd.read_csv(files[3])

fig, ax = plt.subplots(figsize=(22, 11))
ax.plot(df['Time [ns]'], df['Dout'], '.-', label='Original Dout')
ax.legend()

In [ ]:
df = pd.read_csv(files[3])
condition = (df['Time Index'] >= 616) & (df['Time Index'] <= 671)
reduced_df = df.loc[~condition].reset_index(drop=True)
df

In [ ]:
input_array = []
full_array = []

for i in range(1, 9):
    input_array.append(reduced_df.loc[reduced_df['Channel'] == i][['Dout_S1', 'Dout_S2']].to_numpy())
    full_array.append(df.loc[df['Channel'] == i][['Dout_S1', 'Dout_S2']].to_numpy())

In [ ]:
full_data = np.stack(full_array, axis=1)
data = np.stack(input_array, axis=1)
data.shape

In [ ]:


# --- 1. Generate Synthetic Data (Replace with your data loading) ---
# Assuming data has shape (number_of_time_steps, number_of_sources, 2)
# where the last dimension contains [x1, x2] for each source.
# Generate synthetic x1 and x2 data for each source, based on the true data
# Add some noise and variations to simulate real-world data

# Define some 'true' underlying parameters for the synthetic data generation
# These are NOT the parameters we are trying to find with the optimization,
# but rather parameters used to create the synthetic data itself.
# The optimization will try to recover parameters close to these.

num_sources = 8
true_a = np.random.uniform(0.8, 1.2, num_sources)
true_b = np.random.uniform(0.5, 1.5, num_sources)
true_c = np.random.uniform(-0.5, 0.5, num_sources)

# --- Replace the above synthetic data generation with loading your actual data ---
# Example:
# data = np.load('your_data.npy')
# or if it's in a CSV:
# import pandas as pd
# df = pd.read_csv('your_data.csv')
# # You'll need to reshape your data into the (num_time_steps, num_sources, 2) format


# --- 2. Define the function to compute channel values ---
def compute_channel_values(params, data_slice):
    """
    Computes the values for all channels at a single time step.

    Args:
        params (np.ndarray): Flattened array of parameters [a1..a8, b1..b8, c1..c8].
                             Shape (num_sources * 3,).
        data_slice (np.ndarray): Data for a single time step. Shape (num_sources, 2).

    Returns:
        np.ndarray: Computed values for all channels at this time step. Shape (num_sources,).
    """
    num_sources = data_slice.shape[0]
    a_params = params[:num_sources]
    b_params = params[num_sources:2*num_sources]
    c_params = params[2*num_sources:]

    x1 = data_slice[:, 0]
    x2 = data_slice[:, 1]

    # Compute V_i = a_i * (x1_i - b_i * x2_i - c_i)
    computed_values = a_params * (x1 - b_params * x2 - c_params)

    return computed_values

# --- 3. Define the objective function to minimize ---
def objective_function(params, data):
    """
    Computes the total 'disagreement' across all channels over all time steps.
    We minimize the sum of variances of computed values across channels at each time step.

    Args:
        params (np.ndarray): Flattened array of parameters [a1..a8, b1..b8, c1..c8].
                             Shape (num_sources * 3,).
        data (np.ndarray): The full dataset. Shape (num_time_steps, num_sources, 2).

    Returns:
        float: The total sum of variances across time steps.
    """
    num_time_steps = data.shape[0]
    total_variance = 0

    for t in range(num_time_steps):
        data_slice = data[t, :, :] # Data for all channels at time step t
        computed_values_at_t = compute_channel_values(params, data_slice)

        # Calculate the variance of the computed values across the 8 channels at time t
        variance_at_t = np.var(computed_values_at_t)

        total_variance += variance_at_t

    return total_variance

# --- 4. Set initial guesses for the parameters ---
# It's important to provide reasonable initial guesses if possible.
# If you have no idea, starting with 1s and 0s can be a starting point,
# but better guesses can help the optimizer find the global minimum faster.
# The parameters are ordered as [a1..a8, b1..b8, c1..c8]
initial_a_guess = np.ones(num_sources)
initial_b_guess = np.full_like(initial_a_guess, 0.05/5*8.5)
initial_c_guess = np.zeros(num_sources)

initial_params = np.concatenate((initial_a_guess, initial_b_guess, initial_c_guess))

# --- 5. Run the optimization ---
print("Starting optimization...")

# Using the 'Nelder-Mead' method, which is a robust direct search method
# that doesn't require the gradient of the objective function.
# For potentially better performance on larger datasets, other methods like 'L-BFGS-B'
# which can use gradients (if provided, or estimated) might be considered,
# but Nelder-Mead is a good starting point.
result = minimize(objective_function, initial_params, args=(data,), method='Nelder-Mead', options={'disp': True, 'maxiter': 10000})

print("\nOptimization finished.")

# --- 6. Present the results ---
optimized_params = result.x

optimized_a = optimized_params[:num_sources]
optimized_b = optimized_params[num_sources:2*num_sources]
optimized_c = optimized_params[2*num_sources:]

print("\nOptimized Parameters:")
for i in range(num_sources):
    print(f"Channel {i+1}:")
    print(f"  a = {optimized_a[i]:.4f}")
    print(f"  b = {optimized_b[i]:.4f}")
    print(f"  c = {optimized_c[i]:.4f}")

# You can also check the final value of the objective function
print(f"\nFinal objective function value (sum of variances): {result.fun:.4f}")
print(f"Optimization successful: {result.success}")
print(f"Message: {result.message}")

# --- Optional: Verify with synthetic data ---
# If you used the synthetic data generation, you can compare the optimized
# parameters to the 'true' parameters used to generate the data.
if 'true_a' in locals(): # Check if synthetic data variables exist
    print("\nComparison with Synthetic Data 'True' Parameters:")
    for i in range(num_sources):
        print(f"Channel {i+1}:")
        print(f"  Optimized a = {optimized_a[i]:.4f}, True a = {true_a[i]:.4f}")
        print(f"  Optimized b = {optimized_b[i]:.4f}, True b = {true_b[i]:.4f}")
        print(f"  Optimized c = {optimized_c[i]:.4f}, True c = {true_c[i]:.4f}")

In [ ]:
tmp_array = np.array([])
for i in range(128):
    new_Dout = compute_channel_values(optimized_params, full_data[i, :, :])
    tmp_array = np.append(tmp_array, new_Dout)
df['new_Dout'] = tmp_array

In [ ]:
fig, ax = plt.subplots(figsize=(22, 11))

ax.plot(df['Time [ns]'], df['Dout'], '.-', label='Original Dout')
ax.plot(df['Time [ns]'], df['new_Dout'], '.-', label='Recalculated Dout')

ax.legend()